# Lenses and Prisms
(https://blog.jle.im/entry/lenses-products-prisms-sums.html)

Here is the definition of a `Lens'`

In [6]:
:ext ExistentialQuantification
:ext RecordWildCards

import Data.Bifunctor (first)

data Lens' s a = forall q. Lens'
  { split :: s -> (a, q)
  , unsplit :: (a, q) -> s 
  }
  
-- let's use it to implement the lens api
view :: Lens' s a -> (s -> a)
view Lens'{..} = fst . split

set :: Lens' s a -> (s -> a -> s)
set Lens'{..} x v = let (_, q) = split x in unsplit (v, q)

overL :: Lens' s a -> (a -> a) -> (s -> s)
overL Lens'{..} f = unsplit . first f . split

A simple product type that we can write a lenses for

In [30]:
data Person = P
  { _pName :: String
  , _pAge :: Int
  } deriving Show
  
pName = Lens'
  { split = \(P n a) -> (n, a)
  , unsplit = uncurry P
  }

pAge = Lens'
  { split = \(P n a) -> (a, n)
  , unsplit = uncurry $ flip P
  }
  
view pName $ P "Bob" 40
set pName (P "Bob" 40) "Terry"

"Bob"

P {_pName = "Terry", _pAge = 40}

This is to demonstrate that a `Lens` is a witness to one side of a product. If the product is `(a, b)`, then we have two lenses.

Prisms are like Lenses except they represent a Sum rather than a Product

In [8]:
:ext LambdaCase

import Numeric.Natural

data Prism' s a = forall q. Prism'
  { match :: s -> Either a q
  , inject :: Either a q -> s
  }
  
-- implement the prism api
preview :: Prism' s a -> s -> Maybe a
preview Prism'{..} x = case match x of
  Left a -> Just a
  _ -> Nothing
  
review :: Prism' s a -> (a -> s)
review Prism'{..} = inject . Left

overP :: Prism' s a -> (a -> a) -> (s -> s)
overP Prism'{..} f = inject . first f . match

data Shape = Circle Double
           | RegPoly Natural Double
           
_Circle = Prism'
  { match = \case
      Circle r -> Left r
      RegPoly s l -> Right (s, l)
  , inject = \case
      Left r -> Circle r
      Right (s, l) -> RegPoly s l
  }
  
_Prism = Prism'
  { match = \case
      RegPoly s l -> Left (s, l)
      Circle r -> Right r
  , inject = \case
      Left (s, l) -> RegPoly s l
      Right r -> Circle r
  }
  
preview _Circle $ Circle 30

Just 30.0

## Creating Lenses in the Style of the Lens Library
In the book "Haskell Cookbook" there is chapter on lenses that seems to be more in line with the `Lens` package itself. Is the difference that this is more general than the tuple based reprsentation above?

In [1]:
-- a simple type to test on
data Point = Point Double Double deriving Show

Now we define the types for our getter and setter functions. Getter takes a structure `s` and returns a property of it `a`.

Setter takes a structure `s` and a replacement property `b` and returns a `t` which is a modified structure containing property `b`.

In [2]:
type Getter s a = s -> a
type Setter s b t = s -> b -> t

Now we want to combine `Getter` and `Setter` into a single unified type

In [9]:
:ext RankNTypes

type Lens s t a b = forall f. Functor f => (a -> f b) -> s -> f t

This is a Rank 2 type because it's for all functors `f` and `f` is not found on the left side.

This type signature is difficult to understand. Let's see how we construct a member.

In [29]:
lens :: Getter s a -> Setter s b t -> Lens s t a b
lens getter setter f s = fmap (setter s) . f $ getter s

We just need a function `f` which is of type `a -> f b` meaning it takes some component `a` of `s`, morphs it to a `b` inside of some functor `f`.

Then fmapping our partially applied Setter gives us `f t`

Now we will define lenses for our `Point` type. Since we are not changing the data structure at all, we will use a simplified definition of `Lens`.

In [17]:
type Lens' s a = Lens s s a a

x :: Lens' Point Double
x = lens getter setter where
  getter (Point xv _) = xv
  setter (Point _ yv) xv = Point xv yv
  
y :: Lens' Point Double
y = lens getter setter where
  getter (Point _ yv) = yv
  setter (Point xv _) yv = Point xv yv

So to use these lenses we would need to have an `a -> f b` as well as the `Point`. But what should the `f` be? Let's define a const functor for this purpose

In [50]:
:ext DeriveFunctor
:ext StandaloneDeriving

newtype Access a s = Access { access :: a } deriving Show
deriving instance Functor (Access a)

:t Access

Access :: forall a s. a -> Access a s

This provides us with the type of the first argument to a Lens (`a -> f b`). Let's now write a function that gets a field given a lens and a data structure. Notice that since `Access a s` is functorial in `s`, fmapping on it does nothing to the contained `a` because `s` is used as a phantom type. This means that the `fmap (Setter s)` that gets applied will have no effect which is exactly what we want for our `view` function.

In [51]:
view :: Lens' s a -> s -> a
view l = access . l Access

let pt = Point 2 3
view x pt

fmap (7:) $ Access 2
fmap (+4) $ Access 2

2.0

Access {access = 2}

Access {access = 2}

The way this works is that, having been supplied with `a -> f b`, the `Lens'` then consumes the `Point`. It first invokes the getter on the point which produces the value `2.0`. This gets fed into `Access`, so we now have `(Access 2.0)` of type `Access Double Point`. Then `Setter s` is fmapped in, which does not affect the `a` that `Access` is holding since the functor instance is over `Access a`, the `s` in `Access a s` is effectively a phantom type. Fmapping over `Access a s` does nothing, this is why we call it a Const Functor.

Similiarly, let's define a generic function for setting a field using an Identity functor.

In [47]:
newtype Binder a = Binder { bound :: a } deriving Functor

set :: Lens' s a -> a -> s -> s
set l d = bound . l (const (Binder d))

set x 5 pt

Point 5.0 3.0

It makes sense that we have `const (Binder d)` for `f` instead of just `Binder` (like in `view`) because the result from the `getter` within the lens is discarded and `f` passes along the `Binder d` which then has the `setter` applied to it via fmap. This time,`Binder` is not a Const Functor, the setter function is applied to it's contents, which is the `Point`.

## Creating Prisms in the Style of Control.Lens.Prism

In [6]:
-- a sum type we will write prisms for
data Tree a = Leaf a | Branch (Tree a) a (Tree a)

Now we need the machinery for the `Prism` type

In [16]:
:ext RankNTypes
:ext KindSignatures

:m Data.Either

class Profunctor p where
  dimap :: (c -> a) -> (b -> d) -> p a b -> p c d
  dimap f g = lmap f . rmap g
  
  lmap :: (c -> a) -> p a b -> p c b
  lmap f = dimap f id
  
  rmap :: (b -> d) -> p a b -> p a d
  rmap g = dimap id g

class Profunctor p => Choice (p :: * -> * -> *) where
  left' :: p a b -> p (Either a c) (Either b c)
  right' :: p a b -> p (Either c a) (Either c b)  

type Prism s t a b = forall p f. (Choice p, Applicative f) => p a (f b) -> p s (f t)

-- less general prism which does not modify the underlying data type
type Prism' s a = Prism s s a a

-- functions for getting or setting a component within a sum type
type PSetter b t = b -> t
type PGetter s t a = s -> Either t a -- for Prism' this can be Maybe instead of Either

prism :: PSetter b t -> PGetter s t a -> Prism s t a b
prism setter getter = dimap getter (either pure (fmap setter)) . right'
-- note: getter is applied contravariantly here

The `prism` function is used to construct a `Prism`. It works as follows:
- right' from the `Choice` instance transforms the components of the profuctor into `Either`s
- dimap first maps the getter onto the first component of the `Profunctor`, which is contravariant. So mapping `s -> Either t a` onto `p (Either t a) _` produces `p s _`.
- dimap then maps `either pure (fmap setter)` onto the second component of the `Profunctor`. This consumes the `Either c (f b)` to produce an `f t` by relying on the `Applicative` instance of `f` to generate a useable type in the case that `Left c` is encountered or to fmap the setter over `f b` in the `Right` case. 

We can also define `prism'`, which uses `Maybe` instead of `Either` to produce a `Prism'`

In [18]:
prism' :: (a -> s) -> (s -> Maybe a) -> Prism' s a
prism' setter getter = prism setter (\s -> maybe (Left s) Right $ getter s)

Here we are providing a wrapper function around the getter which performs the transformation from `Maybe a` to `Either s a`.

Let's write some getters and setters for our `Tree` type:

TODO: Need an example that's a plain sum, not a sum that contains products.

In [33]:
getLeaf :: Tree a -> Maybe a
getLeaf (Leaf a) = Just a
getLeaf _ = Nothing

setLeaf :: a -> Tree a
setLeaf a = Leaf a

getValue :: Tree a -> Maybe a
getValue (Leaf a) = Just a
getValue (Branch _ a _) = Just a

setValue :: a -> Tree a
setValue a = Leaf a

getLeft :: Tree a -> Maybe (Tree a)
getLeft (Leaf _) = Nothing
getLeft (Branch l _ _) = Just l

setLeft :: Tree a -> Tree a
setLeft l = l--Branch l a r

getRight :: Tree a -> Maybe (Tree a)
getRight (Leaf _) = Nothing
getRight (Branch l a r) = Just r

setRight :: Tree a -> Tree a
setRight r = r--Branch l a r

treeValuePrism :: Prism' (Tree a) a
treeValuePrism = prism' setValue getValue